In [1]:
#if(!require("knitr")) install.packages("knitr")
if(!require("tree")) install.packages("tree")
if(!require("readr")) install.packages("readr")
#if(!require("knitr")) install.packages("knitr")
if(!require("caret")) install.packages("caret")
#if(!require("gridExtra")) install.packages("gridExtra")
if(!require("randomForest")) install.packages("randomForest")
if(!require("foreach")) install.packages("foreach")
if(!require("e1071")) install.packages("e1071")
if(!require("furrr")) install.packages("furrr")
if(!require("parallel")) install.packages("parellel")
if(!require("doParallel")) install.packages("doParallel")




Loading required package: tree

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘tree’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: readr

Loading required package: caret

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘caret’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘data.table’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘recipes’, ‘pROC’


Loading required package: randomForest

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘randomForest’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading 

In [2]:
require("readr")
require("tidyverse")
#require("knitr")
require("tree")
require("caret")
#require("gridExtra")
require("furrr")
require("parallel")
require("doParallel")


Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ dplyr   0.8.5
✔ tibble  3.0.0     ✔ stringr 1.4.0
✔ tidyr   1.0.2     ✔ forcats 0.5.0
✔ purrr   0.3.3     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::when()       masks foreach::when()

Loading required package: tree

Registered S3 method overwritten by 'tree':
  method     from
  print.tree cli 

Loading required package: caret

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: furrr

Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: doParallel

Loading re

In [4]:
# data parsing
soldat <- read_csv("soldat.csv")
soldat$y <- factor(soldat$y,levels = c(-1,1),labels = c("insoluble","soluble"))
n <- nrow(soldat)
p <- ncol(soldat)

Parsed with column specification:
cols(
  .default = col_double()
)

See spec(...) for full column specifications.



In [0]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

In [0]:
v <- cor(soldat %>% select(-y), use = "pairwise.complete.obs")
high_corr <- colnames(soldat)[findCorrelation(v, cutoff = 0.95)]
soldat <- soldat %>% select(-all_of(high_corr))

In [0]:
set.seed(1234)
inTest <- createDataPartition(soldat$y,p = 0.5, list = FALSE)[,1]
test <- soldat[inTest,]
training <- soldat[-inTest,]

In [0]:
control <- trainControl(method="cv", number=5, search="grid",classProbs = TRUE,
allowParallel = TRUE,summaryFunction = twoClassSummary)
metric <- "ROC"
tunegrid <- expand.grid(mtry=seq(2,12,2))
rf_models <- list()

In [13]:
i <- 1
for (ntree in c(1000,1500,2000,2500)) {
  cat(paste0("Training with ", ntree, " trees ..."))
  rf_models[[i]] <- train(y ~ ., 
                                            data = training, 
                                            method = "rf", 
                                            metric = metric, 
                                            tuneGrid = tunegrid, 
                                            trControl = control, 
                                            ntree = ntree)
  i <- i+1
}
save(rf_models,file="rf_models.Rdata")

Training with 1000 trees ...Training with 1500 trees ...Training with 2000 trees ...Training with 2500 trees ...

In [14]:
print(rf_models[[3]])
#1:51

Random Forest 

2815 samples
  46 predictor
   2 classes: 'insoluble', 'soluble' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 2252, 2252, 2251, 2253, 2252 
Resampling results across tuning parameters:

  mtry  ROC        Sens       Spec     
   2    0.8539496  0.9014932  0.5893467
   4    0.8534912  0.8888956  0.5949673
   6    0.8523600  0.8871813  0.6024352
   8    0.8507544  0.8808809  0.6071081
  10    0.8508441  0.8785919  0.6071081
  12    0.8512566  0.8797331  0.6201878

ROC was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.
